In [1]:
!pip install -q transformers accelerate bitsandbytes
!pip install -q PyMuPDF gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 104.0 MB/s eta 0:00:00


In [7]:
from getpass import getpass
hf_token = getpass('Enter your Hugging Face token:')

Enter your Hugging Face token:··········


In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

def generate_response(prompt, max_tokens=512):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=max_tokens, do_sample=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
import fitz

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [10]:
def summarize_text(text):
    prompt = f"Summarize the following document:\n\n{text}\n\nSummary:"
    return generate_response(prompt, max_tokens=256)

def answer_question(text, question):
    prompt = f"You are given a document:\n\n{text}\n\nAnswer the following question based only on the content above:\nQ: {question}\nA:"
    return generate_response(prompt, max_tokens=256)


In [11]:
import gradio as gr

def process_pdf(file, question):
    text = extract_text_from_pdf(file.name)
    summary = summarize_text(text)
    answer = answer_question(text, question)
    return summary, answer

with gr.Blocks() as demo:
    gr.Markdown("### 📄 مساعد Mistral لقراءة وتلخيص ملفات PDF")
    file_input = gr.File(label="Upload PDF file", file_types=[".pdf"])
    question_input = gr.Textbox(label="Ask a question about the file content")
    summary_output = gr.Textbox(label="📋 Summary")
    answer_output = gr.Textbox(label="💬 Answer to the question")
    submit_button = gr.Button("Start analysis")

    submit_button.click(
        fn=process_pdf,
        inputs=[file_input, question_input],
        outputs=[summary_output, answer_output]
    )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2ab8eb224421e1e85f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
